In [1]:
# Import libraries
import pandas as pd
import os
import numpy as np

# Set working directory
os.chdir('C:\\Users\\aaron\\Desktop\\GitHub\\Thesis\\Python')

In [2]:
penalties = {}
for t in range(0, 6):
    penalties[t] = pd.read_excel('Data\Transfermkt\penales.xlsx', sheet_name = t)
    #penalties[t] = penalties[t].dropna(axis = 0)
    penalties[t] = penalties[t][penalties[t]['Unnamed: 5'].notna()]
    penalties[t] = penalties[t][['Date', 'HomeTeam', 'AwayTeam', 'PenAtt']]
    penalties[t] = penalties[t].rename(columns = {'PenAtt' : 'TeamPenAtt'})
    penalties[t]['Partido'] = penalties[t]['HomeTeam'] + ' vs ' + penalties[t]['AwayTeam']
    penalties[t]['Penatt'] = 1
    penalties[t] = penalties[t].groupby(['Date', 'Partido', 'TeamPenAtt', 'HomeTeam', 'AwayTeam']).agg({
        'Penatt' : 'sum'}).reset_index()
    penalties[t]['Year'] = t+1998

In [3]:
penalties = pd.concat(penalties.values(), ignore_index = True)
penalties

,Date,Partido,TeamPenAtt,HomeTeam,AwayTeam,Penatt,Year
0,1998-08-30,RCD Espanyol Barcelona vs CD Tenerife,CD Tenerife,RCD Espanyol Barcelona,CD Tenerife,1,1998
1,1998-09-12,Atlético de Madrid vs UD Salamanca (- 2013),Atlético de Madrid,Atlético de Madrid,UD Salamanca (- 2013),1,1998
2,1998-09-12,Deportivo de La Coruña vs Valencia CF,Deportivo de La Coruña,Deportivo de La Coruña,Valencia CF,1,1998
3,1998-09-20,Real Oviedo vs CD Tenerife,Real Oviedo,Real Oviedo,CD Tenerife,1,1998
4,1998-09-26,Athletic Bilbao vs Real Madrid,Athletic Bilbao,Athletic Bilbao,Real Madrid,1,1998
...,...,...,...,...,...,...,...
581,2004-05-16,Real Murcia CF vs Real Madrid CF,Real Murcia CF,Real Murcia CF,Real Madrid CF,1,2003
582,2004-05-23,Athletic Club vs Atlético de Madrid,Atlético de Madrid,Athletic Club,Atlético de Madrid,1,2003
583,2004-05-23,RC Celta de Vigo vs RCD Mallorca,RC Celta de Vigo,RC Celta de Vigo,RCD Mallorca,1,2003
584,2004-05-23,Real Madrid CF vs Real Sociedad,Real Madrid CF,Real Madrid CF,Real Sociedad,1,2003


In [4]:
def standard_teams(df):
    replacements = {
        'Albacete Balompié' : 'Albacete', 'Athletic Bilbao' : 'Athletic Club', 
        'CA ' : '', 'CD ' : '', 'CF ' : '', 'de ' : '', 
        'Deportivo Alavés' : 'Alavés', 
        ' CF' : '', 'FC ' : '', 
        'RC ' : '', 'REspanyol' : 'Espanyol',
        'Espanyol Barcelona' : 'Espanyol', 'RM' : 'M',
        ' Balompié' : '', 'Real Oviedo' : 'Oviedo', 
        'Real Racing Club ' : 'Racing Santander', 'Real Racing Club' : 'Racing Santander',
        'Real Z' : 'Z', 'Real Valladolid' : 'Valladolid',
        ' Huelva' : '', ' FC' : '', 'UD ' : '',
        'Real Betis' : 'Betis'
    }
    
    variables = ['HomeTeam', 'AwayTeam', 'TeamPenAtt', 'Partido']
    for vars in variables:
        for old_value, new_value in replacements.items():
            df[vars] = df[vars].str.replace(old_value, new_value, regex = True)

# Apply the function
standard_teams(penalties)

In [5]:
variables = ['HomeTeam', 'AwayTeam', 'TeamPenAtt', 'Partido']

for vars in variables:
    penalties[vars] = penalties[vars].str.replace(" (- 2013)", "", regex=False)
    penalties[vars] = penalties[vars].str.replace(" (- 2010)", "", regex=False)

In [6]:
penalties['HomeTotPKatt'] = np.where(penalties['TeamPenAtt'] == penalties['HomeTeam'], 
                                     penalties['Penatt'], 0)

penalties['AwayTotPKatt'] = np.where(penalties['TeamPenAtt'] == penalties['AwayTeam'], 
                                     penalties['Penatt'], 0)

penalties['HomeTotPKcon'] = penalties['AwayTotPKatt']
penalties['AwayTotPKcon'] = penalties['HomeTotPKatt']

penalties = penalties.rename(columns = {'HomeTeam' : 'Home_Team', 'AwayTeam' : 'Away_Team'})

penalties = penalties[['Home_Team', 'Away_Team', 'Partido', 'Year', 'HomeTotPKatt',
                      'AwayTotPKatt', 'HomeTotPKcon', 'AwayTotPKcon']]

In [7]:
penalties = penalties.groupby(['Partido', 'Year', 'Home_Team', 'Away_Team']).agg({
    'HomeTotPKatt': 'sum', 'AwayTotPKatt': 'sum', 'HomeTotPKcon': 'sum', 
    'AwayTotPKcon' : 'sum'}).reset_index()

In [8]:
penalties

,Partido,Year,Home_Team,Away_Team,HomeTotPKatt,AwayTotPKatt,HomeTotPKcon,AwayTotPKcon
0,Alavés vs Athletic Club,1999,Alavés,Athletic Club,1,1,1,1
1,Alavés vs Athletic Club,2000,Alavés,Athletic Club,1,0,0,1
2,Alavés vs Athletic Club,2001,Alavés,Athletic Club,0,1,1,0
3,Alavés vs Barcelona,2001,Alavés,Barcelona,1,0,0,1
4,Alavés vs Betis,1999,Alavés,Betis,1,0,0,1
...,...,...,...,...,...,...,...,...
523,Zaragoza vs Real Sociedad,2003,Zaragoza,Real Sociedad,1,0,0,1
524,Zaragoza vs Sevilla,2003,Zaragoza,Sevilla,1,1,1,1
525,Zaragoza vs Valencia,1999,Zaragoza,Valencia,0,1,1,0
526,Zaragoza vs Villarreal,2001,Zaragoza,Villarreal,1,0,0,1


# SCORED OR MISSED 

In [9]:
typed = ['Scored', 'Missed']

scored = {}
missed = {}

aux = {'Scored': scored, 'Missed': missed}


for it in typed:
    for t in range(0, 10):
        filename = f"{it}.xlsx"
        filepath = os.path.join(r'Data\Transfermkt', filename)  # Use os.path.join to construct the path
        
        aux[it][t] = pd.read_excel(filepath, sheet_name = t)
        aux[it][t] = aux[it][t][aux[it][t]['Unnamed: 5'].notna()]

        aux[it][t]['Year'] = t+2004
        
        aux[it][t] = aux[it][t].rename(columns = {'Result' : 'Home_Team', 'Unnamed: 8' : 'Away_Team', 
                                                  'Penalty taker and Club' : 'PenaltyTaker'})
        
        aux[it][t] = aux[it][t][['Year', 'Matchday', 'PenaltyTaker', 'Home_Team', 'Away_Team']]
        
        aux[it][t]['PenaltyConc'] = np.where(aux[it][t]['Home_Team'] != aux[it][t]['PenaltyTaker'], 
                                     aux[it][t]['Home_Team'], aux[it][t]['Away_Team'])
        
        
        local = ['Home', 'Away']
        misv = ['PKMissed', 'PKconcM']
        scov = ['PKScored', 'PKconcS']

        if it == 'Missed':            
            for im in misv:
                for il in local:
                    var = il + im
                    team = il + "_Team"

                    if im == 'PKMissed':
                        aux[it][t][var] = np.where(aux[it][t][team] == aux[it][t]['PenaltyTaker'], 1, 0)
                    else:
                        aux[it][t][var] = np.where(aux[it][t][team] == aux[it][t]['PenaltyConc'], 1, 0)
        else:
            for iss in scov:
                for il in local:
                    var = il + iss
                    team = il + "_Team"
                    
                    if iss == 'PKScored':
                        aux[it][t][var] = np.where(aux[it][t][team] == aux[it][t]['PenaltyTaker'], 1, 0)
                    else:
                        aux[it][t][var] = np.where(aux[it][t][team] == aux[it][t]['PenaltyConc'], 1, 0)

In [10]:
missed = pd.concat(missed.values(), ignore_index = True)
scored = pd.concat(scored.values(), ignore_index = True)

In [11]:
#Standarize the name of the teams
def standard_teams(df):
    replacements = {
        'Albacete Balompié' : 'Albacete', 'Athletic Bilbao' : 'Athletic Club', 
        'Atlético de Madrid' : 'Atlético Madrid', 'CA Osasuna' : 'Osasuna',
        'Cádiz CF' : 'Cádiz', 'CD Numancia' : 'Numancia', 'CD Tenerife' : 'Tenerife', 
        'Celta de Vigo' : 'Celta Vigo', 'Deportivo Alavés' : 'Alavés',
        'Deportivo de La Coruña' : 'Deportivo La Coruña', 'Elche CF' : 'Elche',
        'FC Barcelona' : 'Barcelona', 'Getafe CF' : 'Getafe',
        'Gimnàstic de Tarragona' : 'Gimnàstic', 'Granada CF' : 'Granada',
        'Hércules CF' : 'Hércules', 'Levante UD' : 'Levante',
        'Málaga CF' : 'Málaga', 'RCD Espanyol Barcelona' : 'Espanyol',
        'RCD Mallorca' : 'Mallorca', 'Real Betis Balompié' : 'Betis',
        'Real Murcia CF' : 'Real Murcia', 'Real Valladolid CF' : 'Valladolid',
        'Real Zaragoza' : 'Zaragoza', 'Recreativo Huelva' : 'Recreativo',
        'Sevilla FC' : 'Sevilla', 'UD Almería' : 'Almería', 'Valencia CF' : 'Valencia',
        'Villarreal CF' : 'Villarreal', 'Xerez CD' : 'Xerez'
    }
    
    variables = ['Home_Team', 'Away_Team', 'PenaltyTaker', 'PenaltyConc']
    for vars in variables:
        for old_value, new_value in replacements.items():
            df[vars] = df[vars].str.replace(old_value, new_value, regex = True)

# Apply the function
standard_teams(missed)
standard_teams(scored)

In [12]:
#Merge Scored and Missed Penalties
conceded = pd.merge(missed, scored, on = ['Year', 'Matchday', 'Home_Team', 'Away_Team', 'PenaltyTaker', 'PenaltyConc'], 
                    how = 'outer')

In [13]:
tosum = ['HomePKScored', 'AwayPKScored', 'HomePKconcS', 'AwayPKconcS',
               'HomePKMissed', 'AwayPKMissed', 'HomePKconcM', 'AwayPKconcM']

conceded = conceded.groupby(['Year', 'Matchday', 'Home_Team', 'Away_Team'])[tosum].sum().reset_index()

In [14]:
# Define variable for the Match information
conceded['Partido'] = conceded['Home_Team'] + " vs " + conceded['Away_Team']

In [15]:
# Adjust wrong info
conceded.loc[(conceded['Partido'] == 'Villarreal vs Valencia') & (conceded['Year'] == 2004), ['AwayPKMissed', 'HomePKconcM']] = [1, 1]
conceded.loc[(conceded['Partido'] == 'Barcelona vs Racing Santander') & (conceded['Year'] == 2005), ['AwayPKMissed', 'HomePKconcM']] = [1, 1]
conceded.loc[(conceded['Partido'] == 'Real Madrid vs Málaga') & (conceded['Year'] == 2008), ['AwayPKMissed', 'HomePKconcM']] = [1, 1]
conceded.loc[(conceded['Partido'] == 'Sevilla vs Athletic Club') & (conceded['Year'] == 2010), ['AwayPKMissed', 'HomePKconcM']] = [1, 1]
conceded.loc[(conceded['Partido'] == 'Real Madrid vs Sevilla') & (conceded['Year'] == 2013), ['AwayPKMissed', 'HomePKconcM']] = [1, 1]

In [16]:
conceded['HomeTotPKcon'] = conceded['HomePKconcS'] + conceded['HomePKconcM']
conceded['AwayTotPKcon'] = conceded['AwayPKconcS'] + conceded['AwayPKconcM']

conceded['HomeTotPKatt'] = conceded['AwayTotPKcon']
conceded['AwayTotPKatt'] = conceded['HomeTotPKcon']

In [17]:
conceded = conceded[['Year', 'Matchday', 'Home_Team', 'Away_Team',
                     'HomeTotPKcon', 'AwayTotPKcon', 'HomeTotPKatt',
                     'AwayTotPKatt', 'Partido']]

NameError: name 'nice' is not defined